In [ ]:
# Library
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import os, cv2, glob, random
import pandas as pd

from numpy import asarray, clip
from tensorflow.keras import regularizers, layers
from tensorflow.keras.applications.resnet50 import ResNet50
import segmentation_models as sm

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.optimizers import adam_v2
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dense
from keras.layers import Dropout, Lambda, Activation, AveragePooling2D, Flatten, Concatenate, GlobalAveragePooling2D, Add
from keras.models import Model, Sequential, load_model
from keras.regularizers import l2, l1


========= MODEL KOMPARASI UNTUK JURNAL 2  ================

In [ ]:
# FCN 8 ===== Long, 2015
def fcn_8(input_shape, num_classes):
    # Input layer
    inputs = Input(shape=input_shape)

    # Encoder
    conv1_1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1_2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1_1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1_2)

    conv2_1 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2_2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2_1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2_2)

    conv3_1 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3_2 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3_1)
    conv3_3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3_2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3_3)

    conv4_1 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4_2 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4_1)
    conv4_3 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4_2)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4_3)

    conv5_1 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5_2 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5_1)
    conv5_3 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5_2)
    pool5 = MaxPooling2D(pool_size=(2, 2))(conv5_3)

    # Decoder
    fc6 = Conv2D(4096, (7, 7), activation='relu', padding='same')(pool5)
    dropout6 = Dropout(rate=0.5)(fc6)

    fc7 = Conv2D(4096, (1, 1), activation='relu', padding='same')(dropout6)
    dropout7 = Dropout(rate=0.5)(fc7)

    score_fr = Conv2D(num_classes, (1, 1), padding='valid')(dropout7)

    score_pool4 = Conv2D(num_classes, (1, 1), padding='valid')(pool4)
    score_pool4c = Conv2DTranspose(num_classes, kernel_size=(4, 4), strides=(2, 2), padding='same')(score_pool4)

    score_pool3 = Conv2D(num_classes, (1, 1), padding='valid')(pool3)
    score_pool3c = Conv2DTranspose(num_classes, kernel_size=(4, 4), strides=(2, 2), padding='same')(score_pool3)
    
    score_pool2 = Conv2D(num_classes, (1, 1), padding='valid')(pool2)
    score_pool2c = Conv2DTranspose(num_classes, kernel_size=(4, 4), strides=(2, 2), padding='same')(score_pool2)
    
    score_pool1 = Conv2D(num_classes, (1, 1), padding='valid')(pool1)
    score_pool1c = Conv2DTranspose(num_classes, kernel_size=(4, 4), strides=(2, 2), padding='same')(score_pool1)
        
    # Output layer
    outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(score_pool1c)

    # Define the model
    modelF = tf.keras.models.Model(inputs=inputs, outputs=outputs)

    return modelF

# modelF = fcn_8(input_shape=(256, 256, 1), num_classes=4)
modelF = fcn_8(input_shape=(512, 512, 3), num_classes=3)
modelF.compile(optimizer='adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])
modelF.summary()

# save the model for future use
# modelF.save('data/model_save/FCN8.hdfs')
# modelF.save('data/model_save/FCN8_256.hdfs')
modelF.save('data/model_save/FCN8_RGB.hdfs')


In [ ]:
#Segnet biasa Badriayanan
def SegNet(input_shape, num_classes):
    # Define the input layer
    inputs = tf.keras.layers.Input(shape=input_shape)

    # Encoder
    enc1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same')(inputs)
    enc1 = tf.keras.layers.BatchNormalization()(enc1)
    enc1 = tf.keras.layers.Activation('relu')(enc1)
    enc2 = tf.keras.layers.MaxPooling2D()(enc1)
    enc2 = tf.keras.layers.Conv2D(128, (3, 3), padding='same')(enc2)
    enc2 = tf.keras.layers.BatchNormalization()(enc2)
    enc2 = tf.keras.layers.Activation('relu')(enc2)
    enc3 = tf.keras.layers.MaxPooling2D()(enc2)
    enc3 = tf.keras.layers.Conv2D(256, (3, 3), padding='same')(enc3)
    enc3 = tf.keras.layers.BatchNormalization()(enc3)
    enc3 = tf.keras.layers.Activation('relu')(enc3)

    # Decoder
    dec3 = tf.keras.layers.Conv2D(256, (3, 3), padding='same')(enc3)
    dec3 = tf.keras.layers.BatchNormalization()(dec3)
    dec3 = tf.keras.layers.Activation('relu')(dec3)
    dec3 = tf.keras.layers.UpSampling2D()(dec3)
    dec2 = tf.keras.layers.Conv2D(128, (3, 3), padding='same')(dec3)
    dec2 = tf.keras.layers.BatchNormalization()(dec2)
    dec2 = tf.keras.layers.Activation('relu')(dec2)
    dec2 = tf.keras.layers.UpSampling2D()(dec2)
    dec1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same')(dec2)
    dec1 = tf.keras.layers.BatchNormalization()(dec1)
    dec1 = tf.keras.layers.Activation('relu')(dec1)

    # Output layer
    outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(dec1)

    # Define the model
    modelS = tf.keras.models.Model(inputs=inputs, outputs=outputs)

    return modelS

# modelS = SegNet(input_shape=(256, 256, 3), num_classes=4)
modelS = SegNet(input_shape=(512, 512, 3), num_classes=3)
modelS.compile(optimizer='adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])
modelS.summary()

# save the model for future use
# modelS.save('data/model_save/Segnet.hdfs')
# modelS.save('data/model_save/Segnet_256.hdfs')
modelS.save('data/model_save/Segnet_RGB.hdfs')


In [ ]:
#AtrousDenseUDeconvNet

def A_UnetDense(input_shape, num_classes):
    inputs = Input(input_shape)
    L2 = BatchNormalization()(inputs)
    L3 = Conv2D(16, (3, 3), activation="relu", padding="same")(L2)
    L4 = Conv2D(8, (3, 3),dilation_rate=2, padding="same")(L3)
    L5 = Conv2D(16, (3, 3),dilation_rate=4, padding="same")(L4)
    L6 = Conv2D(32, (3, 3),dilation_rate=8, padding="same")(L5)
    L7 = Concatenate()([L3, L4, L5, L6])
    L8 = Conv2D(64, (3, 3), activation="relu", padding="same")(L7)
    L9 = Dropout(0.1)(L8)
    L10 = AveragePooling2D(pool_size=(2, 2))(L9)
    L11 = BatchNormalization()(L10)
    L12 = Conv2D(32, (3, 3), dilation_rate=2, padding="same")(L11) 
    L13 = Conv2D(64, (3, 3), dilation_rate=4, padding="same")(L12)
    L14 = Conv2D(128, (3, 3), dilation_rate=8, padding="same")(L13)
    L15 = Concatenate()([L11, L12, L13, L14])
    L16 = Conv2D(256, (3, 3), activation="relu", padding="same")(L15)
    L17 = Dropout(0.1)(L16)
    L18 = AveragePooling2D(pool_size=(2, 2))(L17)
    L19 = BatchNormalization()(L18)
    L20 = Conv2D(128, (3, 3), activation="relu", padding="same")(L19)
    L21 = Dropout(0.1)(L20)
    L22 = MaxPooling2D(pool_size=(2, 2))(L21)
    L23 = Conv2D(256, (3, 3), activation="relu", padding="same")(L22)
    L24 = Dropout(0.1)(L23)
    L25 = MaxPooling2D((2, 2))(L24)
    L26 = Conv2D(512, (3, 3), activation="relu", padding="same")(L25)
    L27 = Dropout(0.1)(L26)
    L28 = MaxPooling2D((2, 2))(L27)
    L29 = Conv2D(1024, (3, 3), activation="relu", padding="same")(L28)
    L30 = Dropout(0.1)(L29)
    L31 = UpSampling2D()(L30)
    L32 = Conv2D(512, (3, 3), activation="relu", padding="same")(L31)
    L33 = Concatenate()([L27, L32])
    L34 = Conv2D(512, (3, 3), activation="relu", padding="same")(L33)
    L35 = UpSampling2D()(L34)
    L36 = Conv2D(256, (3, 3), activation="relu", padding="same")(L35)
    L37 = Concatenate()([L24, L36])
    L38 = Conv2D(256, (3, 3), activation="relu", padding="same")(L37)
    L39 = UpSampling2D()(L38)
    L40 = Conv2D(128, (3, 3), activation="relu", padding="same")(L39)
    L41 = Concatenate()([L21, L40])
    L42 = Conv2D(128, (3, 3), activation="relu", padding="same")(L41)
    Lt1 = UpSampling2D()(L42)
    L43 = Conv2DTranspose(64,(2,2), strides=1, padding='same')(Lt1)
    L44 = Conv2DTranspose(64,(2,2), strides=1, padding='same')(L43)
    Lt2 = UpSampling2D()(L44)
    L45 = Conv2DTranspose(18,(2,2), strides=1, padding='same')(Lt2)
    L46 = Conv2DTranspose(6,(2,2), strides=1, padding='same')(L45)
    
    outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(L46) 
    
    modelA = Model(inputs, outputs, name = 'Model_A_DUDeconvNet')
    return modelA

# modelA = A_UnetDense(input_shape=(256, 256, 1), num_classes=4)
modelA = A_UnetDense(input_shape=(512, 512, 3), num_classes=3)
modelA.summary()
# save the model for future use
# modelA.save('data/model_save/M_ADUDN.hdfs')
# modelA.save('data/model_save/M_ADUDN_256.hdfs')
modelA.save('data/model_save/M_ADUDN_RGB.hdfs')


============= PROGRAM LAIN-LAIN ======================

In [ ]:
def channel_attention(input_feature, name, ratio=8):
    kernel_initializer = tf.contrib.layers.variance_scaling_initializer()
    bias_initializer = tf.constant_initializer(value=0.0)

    with tf.variable_scope(name):
        channel = input_feature.get_shape()[-1]
        avg_pool = tf.reduce_mean(input_feature, axis=[1, 2], keep_dims=True)

        assert avg_pool.get_shape()[1:] == (1, 1, channel)
        avg_pool = tf.layers.dense(inputs=avg_pool,
                                   units=channel // ratio,
                                   activation=tf.nn.relu,
                                   kernel_initializer=kernel_initializer,
                                   bias_initializer=bias_initializer,
                                   name='mlp_0',
                                   reuse=None)
        assert avg_pool.get_shape()[1:] == (1, 1, channel // ratio)
        avg_pool = tf.layers.dense(inputs=avg_pool,
                                   units=channel,
                                   kernel_initializer=kernel_initializer,
                                   bias_initializer=bias_initializer,
                                   name='mlp_1',
                                   reuse=None)
        assert avg_pool.get_shape()[1:] == (1, 1, channel)

        max_pool = tf.reduce_max(input_feature, axis=[1, 2], keep_dims=True)
        assert max_pool.get_shape()[1:] == (1, 1, channel)
        max_pool = tf.layers.dense(inputs=max_pool,
                                   units=channel // ratio,
                                   activation=tf.nn.relu,
                                   name='mlp_0',
                                   reuse=True)
        assert max_pool.get_shape()[1:] == (1, 1, channel // ratio)
        max_pool = tf.layers.dense(inputs=max_pool,
                                   units=channel,
                                   name='mlp_1',
                                   reuse=True)
        assert max_pool.get_shape()[1:] == (1, 1, channel)
        scale = tf.sigmoid(avg_pool + max_pool, 'sigmoid')


    return scale,input_feature * scale

In [ ]:
#load SegNet
def conv_block(input, num_filters): 
    x = Conv2D(num_filters, 3, padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

# Encoder ====
def encoder_2block(input, num_filters):
    x = conv_block(input, num_filters)
    x = conv_block(x num_filters)
    p = MaxPooling2D((2,2))(x)
    return x, p

def encoder_3block(input, num_filters):
    x = conv_block(input, num_filters)
    x = conv_block(x, num_filters)
    x = conv_block(x, num_filters)
    p = MaxPooling2D((2,2))(x)
    return x, p

# Decoder ====
def decoder_3block(input, skip_features, num_filters):
    x = conv_block(input, num_filters)
    x = conv_block(input, num_filters)
    x = conv_block(input, num_filters)
    x = UpSampling2D()(x)    
    return x

def decoder_2block(input, skip_features, num_filters):
    x = conv_block(input, num_filters)
    x = conv_block(input, num_filters)
    x = UpSampling2D()(x)    
    return x

# Build U-Net using the Residu Block
def build_Scattnet(input_shape, n_classes):
    inputs = Input(input_shape)
    
    c1, p1 = encoder_2block(inputs, 64)
    c2, p2 = encoder_2block(p1, 128)
    c3, p3 = encoder_3block(p2, 256)
    c4, p4 = encoder_3block(p3, 512)
    c5, p5 = encoder_3block(p4, 512)
    
    b1 = conv_block(p5, 512) #bridge1
            
    d1 = decoder_3block(b1, c5, 128)
    d2 = decoder_3block(d1, c4, 64)
    d3 = decoder_3block(d2, c3, 32)
    d4 = decoder_1block(d3, c2, 32)
    d5 = decoder_1block(d4, c1, 16)
    
    if n_classes == 1 :
        activation = 'sigmoid'
    else:
        activation = 'softmax'
    
    outputs = Conv2D(n_classes, 1, padding = 'same', activation = activation)(d5) 
    print(activation)
    
    model = Model(inputs, outputs, name = 'ScattNet')
    return model


In [ ]:
#  Load ResNet50 encoder
encoder = tf.keras.applications.ResNet50(input_shape=(512, 512, 3), include_top=False, weights='imagenet')
encoder.summary()

In [ ]:
# Freeze encoder layers
for layer in encoder.layers:
    layer.trainable = False
 
input_decoder = encoder.get_layer('conv5_block3_out').output

# Decoder layers
x = MaxPooling2D((2, 2), padding='same')(input_decoder)
x = Conv2D(8, (3, 3), activation='sigmoid', padding='same')(x)
x = AveragePooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='sigmoid', padding='same')(x) 
#x = UpSampling2D((2, 2))(x)

upconv1 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(x)
conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(upconv1)
conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)
conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)

upconv2 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(upconv2)
conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)
conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

upconv3 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(upconv3)
conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)
conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

upconv4 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(upconv4)
conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)
conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

upconv5 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv9)
conv10 = Conv2D(32, (3, 3), activation='relu', padding='same')(upconv5)
conv10 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv10)
conv10 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv10)

upconv6 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(conv10)
conv11 = Conv2D(16, (3, 3), activation='relu', padding='same')(upconv6)
conv11 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv11)
conv11 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv11)

upconv7 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(conv11)
conv12 = Conv2D(8, (3, 3), activation='relu', padding='same')(upconv7)
conv12 = Conv2D(8, (3, 3), activation='relu', padding='same')(conv12)
conv12 = Conv2D(8, (3, 3), activation='relu', padding='same')(conv12)

# Output layer
output = Conv2D(3, (1, 1), activation='softmax')(conv12)

# Define UNet ResNet50 model
model1 = Model(inputs=encoder.input, outputs=output)

model1.compile(optimizer='adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])
model1.summary()

# save the model for future use
model1.save('data/model_save/ScattNet.hdfs')

================ALTERNATIF MODEL RESNET50 YANG LAIN=======================